In [1]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim, peak_signal_noise_ratio as psnr

from pathlib import Path
from ipywidgets import interact
import os

In [2]:
from torchmetrics.image import MultiScaleStructuralSimilarityIndexMeasure
import torch
from torchvision.transforms.functional import to_tensor
from PIL import Image

In [3]:
ms_ssim = MultiScaleStructuralSimilarityIndexMeasure()

# **Shapes**

In [4]:
imagen = plt.imread('results/p2p_esp_NCCT2DWI/test_latest/images/fake_B/000/test_000_000_fake_B.png')
shape = imagen.shape

print("El shape de la imagen test original es:", shape)

imagen = plt.imread('results/p2p_esp_NCCT2DWI/test_latest/images_grayscale/real_B/000/test_000_000_real_B.png')
shape = imagen.shape

print("El shape de la imagen en escala de grises es:", shape)

El shape de la imagen test original es: (256, 256, 3)
El shape de la imagen en escala de grises es: (256, 256)


In [5]:
# Ruta de paciente en especifico
ruta_real_B_per_paciente = "results/p2p_esp_NCCT2DWI/test_latest/images_grayscale/real_B/022" 
ruta_fake_B_per_paciente = "results/p2p_esp_NCCT2DWI/test_latest/images_grayscale/fake_B/022"

## **Rutas de las imágenes de pacientes en las categorías "real_B" y "fake_B"**

In [6]:
ruta_real_B = "results/p2p_esp_NCCT2DWI/test_latest/images_grayscale/real_B" 
ruta_fake_B = "results/p2p_esp_NCCT2DWI/test_latest/images_grayscale/fake_B"

# **Agrupar Slices de un paciente**

In [7]:
# Obtener la lista de archivos en las rutas
archivos_real_B = sorted(Path(ruta_real_B_per_paciente).glob("*.png"))
archivos_fake_B = sorted(Path(ruta_fake_B_per_paciente).glob("*.png"))

# Cargar las imágenes en listas
imagenes_real_B = [plt.imread(str(f)) for f in archivos_real_B]
imagenes_fake_B = [plt.imread(str(f)) for f in archivos_fake_B]

#verificar dimensiones
for i, img in enumerate(imagenes_real_B):
    assert img.shape == (256, 256), f"Image {i} in real_B has shape {img.shape}"
for i, img in enumerate(imagenes_fake_B):
    assert img.shape == (256, 256), f"Image {i} in fake_B has shape {img.shape}"
    
#Agrupar volumen
volume_real_B = np.stack(imagenes_real_B, axis=-1)
volume_fake_B = np.stack(imagenes_fake_B, axis=-1)

# **CÁLCULO DEL SSIM**

In [8]:
def browse_volume(volume, ssim_value=""):
    def view_image(i):
        plt.imshow(volume[:, :, i], cmap="gray")
        plt.title(ssim_value)
        plt.show()

    interact(view_image, i=(0, volume.shape[2] - 1))

## **SSIM por paciente**

In [9]:
ssim_value = ssim(volume_real_B, volume_fake_B, data_range=volume_real_B.max() - volume_real_B.min())
print(f"Índice de similitud estructural (SSIM) entre real_B y fake_B: {ssim_value:.3f}")

browse_volume(volume_fake_B, ssim_value=f"SSIM: {ssim_value:.3f}")

Índice de similitud estructural (SSIM) entre real_B y fake_B: 0.887


interactive(children=(IntSlider(value=12, description='i', max=24), Output()), _dom_classes=('widget-interact'…

## **Agrupar Slices de todos los pacientes**

In [10]:
# Obtener la lista de carpetas de pacientes
carpetas_pacientes_real_B = sorted(Path(ruta_real_B).glob("*"))
carpetas_pacientes_fake_B = sorted(Path(ruta_fake_B).glob("*"))

ssim_values = []

for carpeta_paciente_real_B, carpeta_paciente_fake_B in zip(carpetas_pacientes_real_B, carpetas_pacientes_fake_B):
    id_paciente = carpeta_paciente_real_B.stem
    archivos_real_B = sorted(carpeta_paciente_real_B.glob("*.png"))
    archivos_fake_B = sorted(carpeta_paciente_fake_B.glob("*.png"))

    imagenes_real_B = [plt.imread(str(f)) for f in archivos_real_B]
    imagenes_fake_B = [plt.imread(str(f)) for f in archivos_fake_B]

    for i, img in enumerate(imagenes_real_B):
        assert img.shape == (256, 256), f"Image {i} in real_B has shape {img.shape}"
    for i, img in enumerate(imagenes_fake_B):
        assert img.shape == (256, 256), f"Image {i} in fake_B has shape {img.shape}"

    volume_real_B = np.stack(imagenes_real_B, axis=-1)
    volume_fake_B = np.stack(imagenes_fake_B, axis=-1)

    ssim_value = ssim(volume_real_B, volume_fake_B, data_range=volume_real_B.max() - volume_real_B.min())
    ssim_values.append(ssim_value)
    print(f"paciente: {id_paciente}, SSIM: {ssim_value:.3f}")

paciente: 000, SSIM: 0.896
paciente: 001, SSIM: 0.895
paciente: 002, SSIM: 0.874
paciente: 003, SSIM: 0.853
paciente: 004, SSIM: 0.889
paciente: 005, SSIM: 0.843
paciente: 006, SSIM: 0.870
paciente: 007, SSIM: 0.903
paciente: 008, SSIM: 0.863
paciente: 009, SSIM: 0.853
paciente: 010, SSIM: 0.889
paciente: 011, SSIM: 0.891
paciente: 012, SSIM: 0.910
paciente: 013, SSIM: 0.894
paciente: 014, SSIM: 0.897
paciente: 015, SSIM: 0.915
paciente: 016, SSIM: 0.889
paciente: 017, SSIM: 0.870
paciente: 018, SSIM: 0.891
paciente: 019, SSIM: 0.875
paciente: 020, SSIM: 0.871
paciente: 021, SSIM: 0.872
paciente: 022, SSIM: 0.887
paciente: 023, SSIM: 0.721
paciente: 024, SSIM: 0.909
paciente: 025, SSIM: 0.869
paciente: 026, SSIM: 0.877
paciente: 027, SSIM: 0.711
paciente: 028, SSIM: 0.904
paciente: 029, SSIM: 0.891
paciente: 030, SSIM: 0.868
paciente: 031, SSIM: 0.859
paciente: 032, SSIM: 0.864
paciente: 033, SSIM: 0.885
paciente: 034, SSIM: 0.891
paciente: 035, SSIM: 0.775
paciente: 036, SSIM: 0.893
p

## **promedio SSIM de todos los pacientes**

In [11]:
promedio_ssim = np.mean(ssim_values)
desviacion_estandar_ssim = np.std(ssim_values)
print(f"SSIM Promedio: {promedio_ssim:.3f}, Desviación Estándar SSIM: {desviacion_estandar_ssim:.4f}")

SSIM Promedio: 0.870, Desviación Estándar SSIM: 0.0436


# **CÁLCULO DEL PSNR**

In [12]:
carpetas_pacientes_real_B = sorted(Path(ruta_real_B).glob("*"))
carpetas_pacientes_fake_B = sorted(Path(ruta_fake_B).glob("*"))

psnr_values = []

for carpeta_paciente_real_B, carpeta_paciente_fake_B in zip(carpetas_pacientes_real_B, carpetas_pacientes_fake_B):
    id_paciente = carpeta_paciente_real_B.stem
    archivos_real_B = sorted(carpeta_paciente_real_B.glob("*.png"))
    archivos_fake_B = sorted(carpeta_paciente_fake_B.glob("*.png"))

    imagenes_real_B = [plt.imread(str(f)) for f in archivos_real_B]
    imagenes_fake_B = [plt.imread(str(f)) for f in archivos_fake_B]

    for i, img in enumerate(imagenes_real_B):
        assert img.shape == (256, 256), f"Image {i} in real_B has shape {img.shape}"
    for i, img in enumerate(imagenes_fake_B):
        assert img.shape == (256, 256), f"Image {i} in fake_B has shape {img.shape}"

    volume_real_B = np.stack(imagenes_real_B, axis=-1)
    volume_fake_B = np.stack(imagenes_fake_B, axis=-1)

    psnr_value = psnr(volume_real_B, volume_fake_B)
    psnr_values.append(psnr_value)
    print(f"paciente: {id_paciente}, PSNR: {psnr_value:.3f}")

paciente: 000, PSNR: 20.741
paciente: 001, PSNR: 21.537
paciente: 002, PSNR: 22.119
paciente: 003, PSNR: 20.580
paciente: 004, PSNR: 22.025
paciente: 005, PSNR: 22.371
paciente: 006, PSNR: 21.690
paciente: 007, PSNR: 22.050
paciente: 008, PSNR: 21.255
paciente: 009, PSNR: 21.426
paciente: 010, PSNR: 21.898
paciente: 011, PSNR: 22.172
paciente: 012, PSNR: 22.875
paciente: 013, PSNR: 23.565
paciente: 014, PSNR: 23.582
paciente: 015, PSNR: 22.907
paciente: 016, PSNR: 23.112
paciente: 017, PSNR: 21.417
paciente: 018, PSNR: 23.018
paciente: 019, PSNR: 22.578
paciente: 020, PSNR: 21.480
paciente: 021, PSNR: 22.034
paciente: 022, PSNR: 22.120
paciente: 023, PSNR: 13.408
paciente: 024, PSNR: 23.227
paciente: 025, PSNR: 20.569
paciente: 026, PSNR: 21.707
paciente: 027, PSNR: 13.249
paciente: 028, PSNR: 22.723
paciente: 029, PSNR: 22.711
paciente: 030, PSNR: 22.046
paciente: 031, PSNR: 21.030
paciente: 032, PSNR: 20.771
paciente: 033, PSNR: 21.773
paciente: 034, PSNR: 22.618
paciente: 035, PSNR:

## **Promedio PSNR de todos los pacientes**

In [13]:
promedio_psnr = np.mean(psnr_values)
desviacion_estandar_psnr = np.std(psnr_values)
print(f"PSNR Promedio: {promedio_psnr:.3f}, Desviación Estándar PSNR: {desviacion_estandar_psnr:.3f}")

PSNR Promedio: 21.372, Desviación Estándar PSNR: 2.407


###### **CÁLCULO DE MS-SSIM**

In [14]:
# Obtener la lista de carpetas de pacientes
carpetas_pacientes_real_B = sorted(Path(ruta_real_B).glob("*"))
carpetas_pacientes_fake_B = sorted(Path(ruta_fake_B).glob("*"))

ms_ssim = MultiScaleStructuralSimilarityIndexMeasure()

ms_ssim_values = []

for carpeta_paciente_real_B, carpeta_paciente_fake_B in zip(carpetas_pacientes_real_B, carpetas_pacientes_fake_B):
    id_paciente = carpeta_paciente_real_B.stem
    archivos_real_B = sorted(carpeta_paciente_real_B.glob("*.png"))
    archivos_fake_B = sorted(carpeta_paciente_fake_B.glob("*.png"))

    imagenes_real_B = [to_tensor(Image.open(str(f))) for f in archivos_real_B]
    imagenes_fake_B = [to_tensor(Image.open(str(f))) for f in archivos_fake_B]

    for i, img in enumerate(imagenes_real_B):
        assert img.shape == (1, 256, 256), f"Image {i} in real_B has shape {img.shape}"
    for i, img in enumerate(imagenes_fake_B):
        assert img.shape == (1, 256, 256), f"Image {i} in fake_B has shape {img.shape}"

    volume_real_B = torch.stack(imagenes_real_B, dim=0)
    volume_fake_B = torch.stack(imagenes_fake_B, dim=0)

    ms_ssim_value = ms_ssim(volume_fake_B, volume_real_B)
    ms_ssim_values.append(ms_ssim_value)  # Agregar valor de MS-SSIM a la lista
    print(f"paciente: {id_paciente}, MS-SSIM: {ms_ssim_value:.3f}")

paciente: 000, MS-SSIM: 0.888
paciente: 001, MS-SSIM: 0.869
paciente: 002, MS-SSIM: 0.895
paciente: 003, MS-SSIM: 0.860
paciente: 004, MS-SSIM: 0.876
paciente: 005, MS-SSIM: 0.850
paciente: 006, MS-SSIM: 0.889
paciente: 007, MS-SSIM: 0.901
paciente: 008, MS-SSIM: 0.865
paciente: 009, MS-SSIM: 0.877
paciente: 010, MS-SSIM: 0.899
paciente: 011, MS-SSIM: 0.882
paciente: 012, MS-SSIM: 0.911
paciente: 013, MS-SSIM: 0.914
paciente: 014, MS-SSIM: 0.899
paciente: 015, MS-SSIM: 0.911
paciente: 016, MS-SSIM: 0.896
paciente: 017, MS-SSIM: 0.891
paciente: 018, MS-SSIM: 0.893
paciente: 019, MS-SSIM: 0.895
paciente: 020, MS-SSIM: 0.888
paciente: 021, MS-SSIM: 0.878
paciente: 022, MS-SSIM: 0.873
paciente: 023, MS-SSIM: 0.657
paciente: 024, MS-SSIM: 0.904
paciente: 025, MS-SSIM: 0.877
paciente: 026, MS-SSIM: 0.893
paciente: 027, MS-SSIM: 0.647
paciente: 028, MS-SSIM: 0.901
paciente: 029, MS-SSIM: 0.897
paciente: 030, MS-SSIM: 0.882
paciente: 031, MS-SSIM: 0.842
paciente: 032, MS-SSIM: 0.837
paciente: 

## **Promedio MS-SSIM de todos los pacientes**

In [15]:
promedio_ms_ssim = np.mean(ms_ssim_values)
desviacion_estandar_ms_ssim = np.std(ms_ssim_values)
print(f"MS-SSIM Promedio: {promedio_ms_ssim:.3f}, Desviación Estándar MS-SSIM: {desviacion_estandar_ms_ssim:.4f}")

MS-SSIM Promedio: 0.866, Desviación Estándar MS-SSIM: 0.0637


## **CREAR NPY**

In [16]:
# Obtener la lista de carpetas de pacientes
carpetas_pacientes_real_B = sorted(Path(ruta_real_B).glob("*"))
carpetas_pacientes_fake_B = sorted(Path(ruta_fake_B).glob("*"))

for carpeta_paciente_real_B, carpeta_paciente_fake_B in zip(carpetas_pacientes_real_B, carpetas_pacientes_fake_B):
    id_paciente = carpeta_paciente_real_B.stem
    archivos_real_B = sorted(carpeta_paciente_real_B.glob("*.png"))
    archivos_fake_B = sorted(carpeta_paciente_fake_B.glob("*.png"))

    imagenes_real_B = [plt.imread(str(f)) for f in archivos_real_B]
    imagenes_fake_B = [plt.imread(str(f)) for f in archivos_fake_B]

    for i, img in enumerate(imagenes_real_B):
        assert img.shape == (256, 256), f"Image {i} in real_B has shape {img.shape}"
    for i, img in enumerate(imagenes_fake_B):
        assert img.shape == (256, 256), f"Image {i} in fake_B has shape {img.shape}"

    volume_real_B = np.stack(imagenes_real_B, axis=-1)
    volume_fake_B = np.stack(imagenes_fake_B, axis=-1)

    # Guardar los volúmenes como archivos numpy (npy)
    np.save(f'volume_real_{id_paciente}.npy', volume_real_B)
    np.save(f'volume_fake_{id_paciente}.npy', volume_fake_B)

# **TAMAÑO DE LESIÓN**

# **Lesiones diminutas (tiny)**

In [17]:
lista_pacientes_tiny = ["000", "007", "010", "011", "012", "014", "025"]

ssim_values = []
psnr_values = []

for id_paciente in lista_pacientes_tiny:
    volume_real_B = np.load(f'volume_real_{id_paciente}.npy')
    volume_fake_B = np.load(f'volume_fake_{id_paciente}.npy')

    ssim_value = ssim(volume_real_B, volume_fake_B, data_range=volume_real_B.max() - volume_real_B.min())
    psnr_value = psnr(volume_real_B, volume_fake_B)

    ssim_values.append(ssim_value)
    psnr_values.append(psnr_value)
0.
promedio_ssim = np.mean(ssim_values)
desviacion_estandar_ssim = np.std(ssim_values)
print(f"SSIM Promedio: {promedio_ssim:.3f}, Desviación Estándar SSIM: {desviacion_estandar_ssim:.3f}")

promedio_psnr = np.mean(psnr_values)
desviacion_estandar_psnr = np.std(psnr_values)
print(f"PSNR Promedio: {promedio_psnr:.3f}, Desviación Estándar PSNR: {desviacion_estandar_psnr:.3f}")

SSIM Promedio: 0.894, Desviación Estándar SSIM: 0.012
PSNR Promedio: 21.984, Desviación Estándar PSNR: 0.996


In [18]:
#Cálculo del MS-SSIM
ms_ssim_values = []

for id_paciente in lista_pacientes_tiny:
    archivos_real_B = sorted(Path(f'results/p2p_esp_NCCT2DWI/test_latest/images_grayscale/real_B/{id_paciente}').glob("*.png"))
    archivos_fake_B = sorted(Path(f'results/p2p_esp_NCCT2DWI/test_latest/images_grayscale/fake_B/{id_paciente}').glob("*.png"))

    imagenes_real_B = [to_tensor(Image.open(str(f))) for f in archivos_real_B]
    imagenes_fake_B = [to_tensor(Image.open(str(f))) for f in archivos_fake_B]

    for i, img in enumerate(imagenes_real_B):
        assert img.shape == (1, 256, 256), f"Image {i} in real_B has shape {img.shape}"
    for i, img in enumerate(imagenes_fake_B):
        assert img.shape == (1, 256, 256), f"Image {i} in fake_B has shape {img.shape}"

    volume_real_B = torch.stack(imagenes_real_B, dim=0)
    volume_fake_B = torch.stack(imagenes_fake_B, dim=0)

    ms_ssim_value = ms_ssim(volume_fake_B, volume_real_B)
    ms_ssim_values.append(ms_ssim_value) 
    #print(f"Paciente: {id_paciente}, MS-SSIM: {ms_ssim_value:.4f}")

# Calcular promedio y desviación estándar para MS-SSIM
promedio_ms_ssim = np.mean(ms_ssim_values)
desviacion_estandar_ms_ssim = np.std(ms_ssim_values)
print(f"MS-SSIM Promedio: {promedio_ms_ssim:.3f}, Desviación Estándar MS-SSIM: {desviacion_estandar_ms_ssim:.3f}")

MS-SSIM Promedio: 0.894, Desviación Estándar MS-SSIM: 0.011


# **Lesiones pequeñas (small)**

In [19]:
lista_pacientes_small = ["002",
"003",
"008",
"013",
"016",
"021",
"023",
"030",
"033"
]

ssim_values = []
psnr_values = []

for id_paciente in lista_pacientes_small:
    volume_real_B = np.load(f'volume_real_{id_paciente}.npy')
    volume_fake_B = np.load(f'volume_fake_{id_paciente}.npy')

    ssim_value = ssim(volume_real_B, volume_fake_B, data_range=volume_real_B.max() - volume_real_B.min())
    psnr_value = psnr(volume_real_B, volume_fake_B)

    ssim_values.append(ssim_value)
    psnr_values.append(psnr_value)

promedio_ssim = np.mean(ssim_values)
desviacion_estandar_ssim = np.std(ssim_values)
print(f"SSIM Promedio: {promedio_ssim:.3f}, Desviación Estándar SSIM: {desviacion_estandar_ssim:.3f}")

promedio_psnr = np.mean(psnr_values)
desviacion_estandar_psnr = np.std(psnr_values)
print(f"PSNR Promedio: {promedio_psnr:.3f}, Desviación Estándar PSNR: {desviacion_estandar_psnr:.3f}")

SSIM Promedio: 0.858, Desviación Estándar SSIM: 0.050
PSNR Promedio: 21.099, Desviación Estándar PSNR: 2.845


In [20]:
#Cálculo del MS-SSIM
ms_ssim_values = []

for id_paciente in lista_pacientes_small:
    archivos_real_B = sorted(Path(f'results/p2p_esp_NCCT2DWI/test_latest/images_grayscale/real_B/{id_paciente}').glob("*.png"))
    archivos_fake_B = sorted(Path(f'results/p2p_esp_NCCT2DWI/test_latest/images_grayscale/fake_B/{id_paciente}').glob("*.png"))

    imagenes_real_B = [to_tensor(Image.open(str(f))) for f in archivos_real_B]
    imagenes_fake_B = [to_tensor(Image.open(str(f))) for f in archivos_fake_B]

    for i, img in enumerate(imagenes_real_B):
        assert img.shape == (1, 256, 256), f"Image {i} in real_B has shape {img.shape}"
    for i, img in enumerate(imagenes_fake_B):
        assert img.shape == (1, 256, 256), f"Image {i} in fake_B has shape {img.shape}"

    volume_real_B = torch.stack(imagenes_real_B, dim=0)
    volume_fake_B = torch.stack(imagenes_fake_B, dim=0)

    ms_ssim_value = ms_ssim(volume_fake_B, volume_real_B)
    ms_ssim_values.append(ms_ssim_value) 
    #print(f"Paciente: {id_paciente}, MS-SSIM: {ms_ssim_value:.4f}")

# Calcular promedio y desviación estándar para MS-SSIM
promedio_ms_ssim = np.mean(ms_ssim_values)
desviacion_estandar_ms_ssim = np.std(ms_ssim_values)
print(f"MS-SSIM Promedio: {promedio_ms_ssim:.3f}, Desviación Estándar MS-SSIM: {desviacion_estandar_ms_ssim:.3f}")

MS-SSIM Promedio: 0.861, Desviación Estándar MS-SSIM: 0.074


# **Lesiones medianas (medium)**

In [21]:
lista_pacientes_medium = ["005",
"006",
"009",
"015",
"017",
"018",
"020",
"026"
]

ssim_values = []
psnr_values = []

for id_paciente in lista_pacientes_medium:
    volume_real_B = np.load(f'volume_real_{id_paciente}.npy')
    volume_fake_B = np.load(f'volume_fake_{id_paciente}.npy')

    ssim_value = ssim(volume_real_B, volume_fake_B, data_range=volume_real_B.max() - volume_real_B.min())
    psnr_value = psnr(volume_real_B, volume_fake_B)

    ssim_values.append(ssim_value)
    psnr_values.append(psnr_value)

promedio_ssim = np.mean(ssim_values)
desviacion_estandar_ssim = np.std(ssim_values)
print(f"SSIM Promedio: {promedio_ssim:.3f}, Desviación Estándar SSIM: {desviacion_estandar_ssim:.3f}")

promedio_psnr = np.mean(psnr_values)
desviacion_estandar_psnr = np.std(psnr_values)
print(f"PSNR Promedio: {promedio_psnr:.3f}, Desviación Estándar PSNR: {desviacion_estandar_psnr:.3f}")

SSIM Promedio: 0.874, Desviación Estándar SSIM: 0.021
PSNR Promedio: 22.002, Desviación Estándar PSNR: 0.624


In [22]:
#Cálculo del MS-SSIM
ms_ssim_values = []

for id_paciente in lista_pacientes_medium:
    archivos_real_B = sorted(Path(f'results/p2p_esp_NCCT2DWI/test_latest/images_grayscale/real_B/{id_paciente}').glob("*.png"))
    archivos_fake_B = sorted(Path(f'results/p2p_esp_NCCT2DWI/test_latest/images_grayscale/fake_B/{id_paciente}').glob("*.png"))

    imagenes_real_B = [to_tensor(Image.open(str(f))) for f in archivos_real_B]
    imagenes_fake_B = [to_tensor(Image.open(str(f))) for f in archivos_fake_B]

    for i, img in enumerate(imagenes_real_B):
        assert img.shape == (1, 256, 256), f"Image {i} in real_B has shape {img.shape}"
    for i, img in enumerate(imagenes_fake_B):
        assert img.shape == (1, 256, 256), f"Image {i} in fake_B has shape {img.shape}"

    volume_real_B = torch.stack(imagenes_real_B, dim=0)
    volume_fake_B = torch.stack(imagenes_fake_B, dim=0)

    ms_ssim_value = ms_ssim(volume_fake_B, volume_real_B)
    ms_ssim_values.append(ms_ssim_value) 
    #print(f"Paciente: {id_paciente}, MS-SSIM: {ms_ssim_value:.4f}")

# Calcular promedio y desviación estándar para MS-SSIM
promedio_ms_ssim = np.mean(ms_ssim_values)
desviacion_estandar_ms_ssim = np.std(ms_ssim_values)
print(f"MS-SSIM Promedio: {promedio_ms_ssim:.3f}, Desviación Estándar MS-SSIM: {desviacion_estandar_ms_ssim:.3f}")

MS-SSIM Promedio: 0.886, Desviación Estándar MS-SSIM: 0.016


# **Lesiones grandes (large)**

In [23]:
lista_pacientes_large = ["019",
"022",
"028",
"029",
"031",
"032",
"034",
"036",
"037"
]

ssim_values = []
psnr_values = []

for id_paciente in lista_pacientes_large:
    volume_real_B = np.load(f'volume_real_{id_paciente}.npy')
    volume_fake_B = np.load(f'volume_fake_{id_paciente}.npy')

    ssim_value = ssim(volume_real_B, volume_fake_B, data_range=volume_real_B.max() - volume_real_B.min())
    psnr_value = psnr(volume_real_B, volume_fake_B)

    ssim_values.append(ssim_value)
    psnr_values.append(psnr_value)

promedio_ssim = np.mean(ssim_values)
desviacion_estandar_ssim = np.std(ssim_values)
print(f"SSIM Promedio: {promedio_ssim:.3f}, Desviación Estándar SSIM: {desviacion_estandar_ssim:.3f}")

promedio_psnr = np.mean(psnr_values)
desviacion_estandar_psnr = np.std(psnr_values)
print(f"PSNR Promedio: {promedio_psnr:.3f}, Desviación Estándar PSNR: {desviacion_estandar_psnr:.3f}")

SSIM Promedio: 0.880, Desviación Estándar SSIM: 0.017
PSNR Promedio: 21.984, Desviación Estándar PSNR: 1.044


In [24]:
#Cálculo del MS-SSIM
ms_ssim_values = []

for id_paciente in lista_pacientes_large:
    archivos_real_B = sorted(Path(f'results/p2p_esp_NCCT2DWI/test_latest/images_grayscale/real_B/{id_paciente}').glob("*.png"))
    archivos_fake_B = sorted(Path(f'results/p2p_esp_NCCT2DWI/test_latest/images_grayscale/fake_B/{id_paciente}').glob("*.png"))

    imagenes_real_B = [to_tensor(Image.open(str(f))) for f in archivos_real_B]
    imagenes_fake_B = [to_tensor(Image.open(str(f))) for f in archivos_fake_B]

    for i, img in enumerate(imagenes_real_B):
        assert img.shape == (1, 256, 256), f"Image {i} in real_B has shape {img.shape}"
    for i, img in enumerate(imagenes_fake_B):
        assert img.shape == (1, 256, 256), f"Image {i} in fake_B has shape {img.shape}"

    volume_real_B = torch.stack(imagenes_real_B, dim=0)
    volume_fake_B = torch.stack(imagenes_fake_B, dim=0)

    ms_ssim_value = ms_ssim(volume_fake_B, volume_real_B)
    ms_ssim_values.append(ms_ssim_value) 
    #print(f"Paciente: {id_paciente}, MS-SSIM: {ms_ssim_value:.4f}")

# Calcular promedio y desviación estándar para MS-SSIM
promedio_ms_ssim = np.mean(ms_ssim_values)
desviacion_estandar_ms_ssim = np.std(ms_ssim_values)
print(f"MS-SSIM Promedio: {promedio_ms_ssim:.3f}, Desviación Estándar MS-SSIM: {desviacion_estandar_ms_ssim:.3f}")

MS-SSIM Promedio: 0.873, Desviación Estándar MS-SSIM: 0.028


# **Sin lesión (no_lesion)**

In [25]:
lista_pacientes_no_lesion = ["001",
"004",0.
"024"
]

ssim_values = []
psnr_values = []

for id_paciente in lista_pacientes_no_lesion:
    volume_real_B = np.load(f'volume_real_{id_paciente}.npy')
    volume_fake_B = np.load(f'volume_fake_{id_paciente}.npy')

    ssim_value = ssim(volume_real_B, volume_fake_B, data_range=volume_real_B.max() - volume_real_B.min())
    psnr_value = psnr(volume_real_B, volume_fake_B)

    ssim_values.append(ssim_value)
    psnr_values.append(psnr_value)

promedio_ssim = np.mean(ssim_values)
desviacion_estandar_ssim = np.std(ssim_values)
print(f"SSIM Promedio: {promedio_ssim:.3f}, Desviación Estándar SSIM: {desviacion_estandar_ssim:.3f}")

promedio_psnr = np.mean(psnr_values)
desviacion_estandar_psnr = np.std(psnr_values)
print(f"PSNR Promedio: {promedio_psnr:.3f}, Desviación Estándar PSNR: {desviacion_estandar_psnr:.3f}")

SSIM Promedio: 0.898, Desviación Estándar SSIM: 0.008
PSNR Promedio: 22.263, Desviación Estándar PSNR: 0.710


In [26]:
#Cálculo del MS-SSIM
ms_ssim_values = []

for id_paciente in lista_pacientes_no_lesion:
    archivos_real_B = sorted(Path(f'results/p2p_esp_NCCT2DWI/test_latest/images_grayscale/real_B/{id_paciente}').glob("*.png"))
    archivos_fake_B = sorted(Path(f'results/p2p_esp_NCCT2DWI/test_latest/images_grayscale/fake_B/{id_paciente}').glob("*.png"))

    imagenes_real_B = [to_tensor(Image.open(str(f))) for f in archivos_real_B]
    imagenes_fake_B = [to_tensor(Image.open(str(f))) for f in archivos_fake_B]

    for i, img in enumerate(imagenes_real_B):
        assert img.shape == (1, 256, 256), f"Image {i} in real_B has shape {img.shape}"
    for i, img in enumerate(imagenes_fake_B):
        assert img.shape == (1, 256, 256), f"Image {i} in fake_B has shape {img.shape}"

    volume_real_B = torch.stack(imagenes_real_B, dim=0)
    volume_fake_B = torch.stack(imagenes_fake_B, dim=0)

    ms_ssim_value = ms_ssim(volume_fake_B, volume_real_B)
    ms_ssim_values.append(ms_ssim_value) 
    #print(f"Paciente: {id_paciente}, MS-SSIM: {ms_ssim_value:.4f}")

# Calcular promedio y desviación estándar para MS-SSIM
promedio_ms_ssim = np.mean(ms_ssim_values)
desviacion_estandar_ms_ssim = np.std(ms_ssim_values)
print(f"MS-SSIM Promedio: {promedio_ms_ssim:.3f}, Desviación Estándar MS-SSIM: {desviacion_estandar_ms_ssim:.3f}")

MS-SSIM Promedio: 0.883, Desviación Estándar MS-SSIM: 0.015


# **VISIBILIDAD DE LESIONES**

# **Es visible**

In [27]:
lista_pacientes_visible = ["003",
"005",
"006",
"007",
"009",
"011",
"015",
"017",
"018",
"019",
"020",
"021",
"022",
"029",
"030",
"031",
"034",
"037"
]

ssim_values = []
psnr_values = []

for id_paciente in lista_pacientes_visible:
    volume_real_B = np.load(f'volume_real_{id_paciente}.npy')
    volume_fake_B = np.load(f'volume_fake_{id_paciente}.npy')

    ssim_value = ssim(volume_real_B, volume_fake_B, data_range=volume_real_B.max() - volume_real_B.min())
    psnr_value = psnr(volume_real_B, volume_fake_B)

    ssim_values.append(ssim_value)
    psnr_values.append(psnr_value)

promedio_ssim = np.mean(ssim_values)
desviacion_estandar_ssim = np.std(ssim_values)
print(f"SSIM Promedio: {promedio_ssim:.3f}, Desviación Estándar SSIM: {desviacion_estandar_ssim:.3f}")

promedio_psnr = np.mean(psnr_values)
desviacion_estandar_psnr = np.std(psnr_values)
print(f"PSNR Promedio: {promedio_psnr:.3f}, Desviación Estándar PSNR: {desviacion_estandar_psnr:.3f}")

SSIM Promedio: 0.875, Desviación Estándar SSIM: 0.019
PSNR Promedio: 21.903, Desviación Estándar PSNR: 0.783


In [28]:
#Cálculo del MS-SSIM
ms_ssim_values = []

for id_paciente in lista_pacientes_visible:
    archivos_real_B = sorted(Path(f'results/p2p_esp_NCCT2DWI/test_latest/images_grayscale/real_B/{id_paciente}').glob("*.png"))
    archivos_fake_B = sorted(Path(f'results/p2p_esp_NCCT2DWI/test_latest/images_grayscale/fake_B/{id_paciente}').glob("*.png"))

    imagenes_real_B = [to_tensor(Image.open(str(f))) for f in archivos_real_B]
    imagenes_fake_B = [to_tensor(Image.open(str(f))) for f in archivos_fake_B]

    for i, img in enumerate(imagenes_real_B):
        assert img.shape == (1, 256, 256), f"Image {i} in real_B has shape {img.shape}"
    for i, img in enumerate(imagenes_fake_B):
        assert img.shape == (1, 256, 256), f"Image {i} in fake_B has shape {img.shape}"

    volume_real_B = torch.stack(imagenes_real_B, dim=0)
    volume_fake_B = torch.stack(imagenes_fake_B, dim=0)

    ms_ssim_value = ms_ssim(volume_fake_B, volume_real_B)
    ms_ssim_values.append(ms_ssim_value) 
    #print(f"Paciente: {id_paciente}, MS-SSIM: {ms_ssim_value:.4f}")

# Calcular promedio y desviación estándar para MS-SSIM
promedio_ms_ssim = np.mean(ms_ssim_values)
desviacion_estandar_ms_ssim = np.std(ms_ssim_values)
print(f"MS-SSIM Promedio: {promedio_ms_ssim:.3f}, Desviación Estándar MS-SSIM: {desviacion_estandar_ms_ssim:.3f}")

MS-SSIM Promedio: 0.879, Desviación Estándar MS-SSIM: 0.021


# **NO es visible**

In [29]:
lista_pacientes_no_visible = ["000",
"002",
"008",
"010",
"012",
"013",
"014",
"016",
"025",
"026",
"027",
"028",
"032",
"033",
"035",
"036"
]

ssim_values = []
psnr_values = []

for id_paciente in lista_pacientes_no_visible:
    volume_real_B = np.load(f'volume_real_{id_paciente}.npy')
    volume_fake_B = np.load(f'volume_fake_{id_paciente}.npy')

    ssim_value = ssim(volume_real_B, volume_fake_B, data_range=volume_real_B.max() - volume_real_B.min())
    psnr_value = psnr(volume_real_B, volume_fake_B)

    ssim_values.append(ssim_value)
    psnr_values.append(psnr_value)

promedio_ssim = np.mean(ssim_values)
desviacion_estandar_ssim = np.std(ssim_values)
print(f"SSIM Promedio: {promedio_ssim:.3f}, Desviación Estándar SSIM: {desviacion_estandar_ssim:.3f}")

promedio_psnr = np.mean(psnr_values)
desviacion_estandar_psnr = np.std(psnr_values)
print(f"PSNR Promedio: {promedio_psnr:.3f}, Desviación Estándar PSNR: {desviacion_estandar_psnr:.3f}")

SSIM Promedio: 0.868, Desviación Estándar SSIM: 0.050
PSNR Promedio: 21.105, Desviación Estándar PSNR: 2.912


In [30]:
#Cálculo del MS-SSIM
ms_ssim_values = []

for id_paciente in lista_pacientes_no_visible:
    archivos_real_B = sorted(Path(f'results/p2p_esp_NCCT2DWI/test_latest/images_grayscale/real_B/{id_paciente}').glob("*.png"))
    archivos_fake_B = sorted(Path(f'results/p2p_esp_NCCT2DWI/test_latest/images_grayscale/fake_B/{id_paciente}').glob("*.png"))

    imagenes_real_B = [to_tensor(Image.open(str(f))) for f in archivos_real_B]
    imagenes_fake_B = [to_tensor(Image.open(str(f))) for f in archivos_fake_B]

    for i, img in enumerate(imagenes_real_B):
        assert img.shape == (1, 256, 256), f"Image {i} in real_B has shape {img.shape}"
    for i, img in enumerate(imagenes_fake_B):
        assert img.shape == (1, 256, 256), f"Image {i} in fake_B has shape {img.shape}"

    volume_real_B = torch.stack(imagenes_real_B, dim=0)
    volume_fake_B = torch.stack(imagenes_fake_B, dim=0)

    ms_ssim_value = ms_ssim(volume_fake_B, volume_real_B)
    ms_ssim_values.append(ms_ssim_value) 
    #print(f"Paciente: {id_paciente}, MS-SSIM: {ms_ssim_value:.4f}")

# Calcular promedio y desviación estándar para MS-SSIM
promedio_ms_ssim = np.mean(ms_ssim_values)
desviacion_estandar_ms_ssim = np.std(ms_ssim_values)
print(f"MS-SSIM Promedio: {promedio_ms_ssim:.3f}, Desviación Estándar MS-SSIM: {desviacion_estandar_ms_ssim:.3f}")

MS-SSIM Promedio: 0.862, Desviación Estándar MS-SSIM: 0.078


# **TIEMPO DESDE EL INICIO DE LOS SÍNTOMAS**

# **Wake-up (desconocido)**

In [31]:
lista_pacientes_desc = ["005",
"014",
"024",
"030",
"034"
]

ssim_values = []
psnr_values = []

for id_paciente in lista_pacientes_desc:
    volume_real_B = np.load(f'volume_real_{id_paciente}.npy')
    volume_fake_B = np.load(f'volume_fake_{id_paciente}.npy')

    ssim_value = ssim(volume_real_B, volume_fake_B, data_range=volume_real_B.max() - volume_real_B.min())
    psnr_value = psnr(volume_real_B, volume_fake_B)

    ssim_values.append(ssim_value)
    psnr_values.append(psnr_value)

promedio_ssim = np.mean(ssim_values)
desviacion_estandar_ssim = np.std(ssim_values)
print(f"SSIM Promedio: {promedio_ssim:.3f}, Desviación Estándar SSIM: {desviacion_estandar_ssim:.3f}")

promedio_psnr = np.mean(psnr_values)
desviacion_estandar_psnr = np.std(psnr_values)
print(f"PSNR Promedio: {promedio_psnr:.3f}, Desviación Estándar PSNR: {desviacion_estandar_psnr:.3f}")

SSIM Promedio: 0.882, Desviación Estándar SSIM: 0.023
PSNR Promedio: 22.769, Desviación Estándar PSNR: 0.561


In [32]:
#Cálculo del MS-SSIM
ms_ssim_values = []

for id_paciente in lista_pacientes_desc:
    archivos_real_B = sorted(Path(f'results/p2p_esp_NCCT2DWI/test_latest/images_grayscale/real_B/{id_paciente}').glob("*.png"))
    archivos_fake_B = sorted(Path(f'results/p2p_esp_NCCT2DWI/test_latest/images_grayscale/fake_B/{id_paciente}').glob("*.png"))

    imagenes_real_B = [to_tensor(Image.open(str(f))) for f in archivos_real_B]
    imagenes_fake_B = [to_tensor(Image.open(str(f))) for f in archivos_fake_B]

    for i, img in enumerate(imagenes_real_B):
        assert img.shape == (1, 256, 256), f"Image {i} in real_B has shape {img.shape}"
    for i, img in enumerate(imagenes_fake_B):
        assert img.shape == (1, 256, 256), f"Image {i} in fake_B has shape {img.shape}"

    volume_real_B = torch.stack(imagenes_real_B, dim=0)
    volume_fake_B = torch.stack(imagenes_fake_B, dim=0)

    ms_ssim_value = ms_ssim(volume_fake_B, volume_real_B)
    ms_ssim_values.append(ms_ssim_value) 
    #print(f"Paciente: {id_paciente}, MS-SSIM: {ms_ssim_value:.4f}")

# Calcular promedio y desviación estándar para MS-SSIM
promedio_ms_ssim = np.mean(ms_ssim_values)
desviacion_estandar_ms_ssim = np.std(ms_ssim_values)
print(f"MS-SSIM Promedio: {promedio_ms_ssim:.3f}, Desviación Estándar MS-SSIM: {desviacion_estandar_ms_ssim:.3f}")

MS-SSIM Promedio: 0.884, Desviación Estándar MS-SSIM: 0.019


# **Early hyperacute (0-6h)***

In [33]:
lista_pacientes_early = ["000",
"001",
"003",
"008",
"013",
"017",
"018",
"022",
"026",
"028",
"029",
"031",
"032",
"033",
"036"
]

ssim_values = []
psnr_values = []

for id_paciente in lista_pacientes_early:
    volume_real_B = np.load(f'volume_real_{id_paciente}.npy')
    volume_fake_B = np.load(f'volume_fake_{id_paciente}.npy')

    ssim_value = ssim(volume_real_B, volume_fake_B, data_range=volume_real_B.max() - volume_real_B.min())
    psnr_value = psnr(volume_real_B, volume_fake_B)

    ssim_values.append(ssim_value)
    psnr_values.append(psnr_value)

promedio_ssim = np.mean(ssim_values)
desviacion_estandar_ssim = np.std(ssim_values)
print(f"SSIM Promedio: {promedio_ssim:.3f}, Desviación Estándar SSIM: {desviacion_estandar_ssim:.3f}")

promedio_psnr = np.mean(psnr_values)
desviacion_estandar_psnr = np.std(psnr_values)
print(f"PSNR Promedio: {promedio_psnr:.3f}, Desviación Estándar PSNR: {desviacion_estandar_psnr:.3f}")

SSIM Promedio: 0.882, Desviación Estándar SSIM: 0.015
PSNR Promedio: 21.882, Desviación Estándar PSNR: 0.941


In [34]:
#Cálculo del MS-SSIM
ms_ssim_values = []

for id_paciente in lista_pacientes_early:
    archivos_real_B = sorted(Path(f'results/p2p_esp_NCCT2DWI/test_latest/images_grayscale/real_B/{id_paciente}').glob("*.png"))
    archivos_fake_B = sorted(Path(f'results/p2p_esp_NCCT2DWI/test_latest/images_grayscale/fake_B/{id_paciente}').glob("*.png"))

    imagenes_real_B = [to_tensor(Image.open(str(f))) for f in archivos_real_B]
    imagenes_fake_B = [to_tensor(Image.open(str(f))) for f in archivos_fake_B]

    for i, img in enumerate(imagenes_real_B):
        assert img.shape == (1, 256, 256), f"Image {i} in real_B has shape {img.shape}"
    for i, img in enumerate(imagenes_fake_B):
        assert img.shape == (1, 256, 256), f"Image {i} in fake_B has shape {img.shape}"

    volume_real_B = torch.stack(imagenes_real_B, dim=0)
    volume_fake_B = torch.stack(imagenes_fake_B, dim=0)

    ms_ssim_value = ms_ssim(volume_fake_B, volume_real_B)
    ms_ssim_values.append(ms_ssim_value) 
    #print(f"Paciente: {id_paciente}, MS-SSIM: {ms_ssim_value:.4f}")

# Calcular promedio y desviación estándar para MS-SSIM
promedio_ms_ssim = np.mean(ms_ssim_values)
desviacion_estandar_ms_ssim = np.std(ms_ssim_values)
print(f"MS-SSIM Promedio: {promedio_ms_ssim:.3f}, Desviación Estándar MS-SSIM: {desviacion_estandar_ms_ssim:.3f}")

MS-SSIM Promedio: 0.881, Desviación Estándar MS-SSIM: 0.022


# **Late hyperacute (6-24h)**


In [35]:
lista_pacientes_late = ["002",
"004",
"006",
"007",
"009",
"010",
"011",
"012",
"015",
"020",
"021",
"037"
]

ssim_values = []
psnr_values = []

for id_paciente in lista_pacientes_late:
    volume_real_B = np.load(f'volume_real_{id_paciente}.npy')
    volume_fake_B = np.load(f'volume_fake_{id_paciente}.npy')

    ssim_value = ssim(volume_real_B, volume_fake_B, data_range=volume_real_B.max() - volume_real_B.min())
    psnr_value = psnr(volume_real_B, volume_fake_B)

    ssim_values.append(ssim_value)
    psnr_values.append(psnr_value)

promedio_ssim = np.mean(ssim_values)
desviacion_estandar_ssim = np.std(ssim_values)
print(f"SSIM Promedio: {promedio_ssim:.3f}, Desviación Estándar SSIM: {desviacion_estandar_ssim:.3f}")

promedio_psnr = np.mean(psnr_values)
desviacion_estandar_psnr = np.std(psnr_values)
print(f"PSNR Promedio: {promedio_psnr:.3f}, Desviación Estándar PSNR: {desviacion_estandar_psnr:.3f}")

SSIM Promedio: 0.883, Desviación Estándar SSIM: 0.020
PSNR Promedio: 21.891, Desviación Estándar PSNR: 0.716


In [36]:
#Cálculo del MS-SSIM
ms_ssim_values = []

for id_paciente in lista_pacientes_late:
    archivos_real_B = sorted(Path(f'results/p2p_esp_NCCT2DWI/test_latest/images_grayscale/real_B/{id_paciente}').glob("*.png"))
    archivos_fake_B = sorted(Path(f'results/p2p_esp_NCCT2DWI/test_latest/images_grayscale/fake_B/{id_paciente}').glob("*.png"))

    imagenes_real_B = [to_tensor(Image.open(str(f))) for f in archivos_real_B]
    imagenes_fake_B = [to_tensor(Image.open(str(f))) for f in archivos_fake_B]

    for i, img in enumerate(imagenes_real_B):
        assert img.shape == (1, 256, 256), f"Image {i} in real_B has shape {img.shape}"
    for i, img in enumerate(imagenes_fake_B):
        assert img.shape == (1, 256, 256), f"Image {i} in fake_B has shape {img.shape}"

    volume_real_B = torch.stack(imagenes_real_B, dim=0)
    volume_fake_B = torch.stack(imagenes_fake_B, dim=0)

    ms_ssim_value = ms_ssim(volume_fake_B, volume_real_B)
    ms_ssim_values.append(ms_ssim_value) 
    #print(f"Paciente: {id_paciente}, MS-SSIM: {ms_ssim_value:.4f}")

# Calcular promedio y desviación estándar para MS-SSIM
promedio_ms_ssim = np.mean(ms_ssim_values)
desviacion_estandar_ms_ssim = np.std(ms_ssim_values)
print(f"MS-SSIM Promedio: {promedio_ms_ssim:.3f}, Desviación Estándar MS-SSIM: {desviacion_estandar_ms_ssim:.3f}")

MS-SSIM Promedio: 0.886, Desviación Estándar MS-SSIM: 0.022


# **Acute (24-96)**


In [37]:
lista_pacientes_acute = ["016",
"019",
"023",
"025"
]

ssim_values = []
psnr_values = []

for id_paciente in lista_pacientes_acute:
    volume_real_B = np.load(f'volume_real_{id_paciente}.npy')
    volume_fake_B = np.load(f'volume_fake_{id_paciente}.npy')

    ssim_value = ssim(volume_real_B, volume_fake_B, data_range=volume_real_B.max() - volume_real_B.min())
    psnr_value = psnr(volume_real_B, volume_fake_B)

    ssim_values.append(ssim_value)
    psnr_values.append(psnr_value)

promedio_ssim = np.mean(ssim_values)
desviacion_estandar_ssim = np.std(ssim_values)
print(f"SSIM Promedio: {promedio_ssim:.3f}, Desviación Estándar SSIM: {desviacion_estandar_ssim:.3f}")

promedio_psnr = np.mean(psnr_values)
desviacion_estandar_psnr = np.std(psnr_values)
print(f"PSNR Promedio: {promedio_psnr:.3f}, Desviación Estándar PSNR: {desviacion_estandar_psnr:.3f}")

SSIM Promedio: 0.838, Desviación Estándar SSIM: 0.068
PSNR Promedio: 19.917, Desviación Estándar PSNR: 3.876


In [38]:
#Cálculo del MS-SSIM
ms_ssim_values = []

for id_paciente in lista_pacientes_acute:
    archivos_real_B = sorted(Path(f'results/p2p_esp_NCCT2DWI/test_latest/images_grayscale/real_B/{id_paciente}').glob("*.png"))
    archivos_fake_B = sorted(Path(f'results/p2p_esp_NCCT2DWI/test_latest/images_grayscale/fake_B/{id_paciente}').glob("*.png"))

    imagenes_real_B = [to_tensor(Image.open(str(f))) for f in archivos_real_B]
    imagenes_fake_B = [to_tensor(Image.open(str(f))) for f in archivos_fake_B]

    for i, img in enumerate(imagenes_real_B):
        assert img.shape == (1, 256, 256), f"Image {i} in real_B has shape {img.shape}"
    for i, img in enumerate(imagenes_fake_B):
        assert img.shape == (1, 256, 256), f"Image {i} in fake_B has shape {img.shape}"

    volume_real_B = torch.stack(imagenes_real_B, dim=0)
    volume_fake_B = torch.stack(imagenes_fake_B, dim=0)

    ms_ssim_value = ms_ssim(volume_fake_B, volume_real_B)
    ms_ssim_values.append(ms_ssim_value) 
    #print(f"Paciente: {id_paciente}, MS-SSIM: {ms_ssim_value:.4f}")

# Calcular promedio y desviación estándar para MS-SSIM
promedio_ms_ssim = np.mean(ms_ssim_values)
desviacion_estandar_ms_ssim = np.std(ms_ssim_values)
print(f"MS-SSIM Promedio: {promedio_ms_ssim:.3f}, Desviación Estándar MS-SSIM: {desviacion_estandar_ms_ssim:.3f}")

MS-SSIM Promedio: 0.831, Desviación Estándar MS-SSIM: 0.101


## **FID (Frequent Inception Distance)**

In [ ]:
imagen_real = Image.open('results/p2p_basic_NCCT2ADC/test_latest/images/real_B/022/test_022_011_real_B.png')
imagen_fake = Image.open('results/p2p_basic_NCCT2ADC/test_latest/images/fake_B/022/test_022_011_fake_B.png')